In [94]:
from caloGraphNN_keras import GravNet
import uproot
import awkward as ak
import numpy as np
import tensorflow.keras.backend as K

In [2]:
model = GravNet(5,5,5,5,"find_phi")

In [105]:
print(model)

In [22]:
data = uproot.open('bstophiphi_gnn_total.root')['L1TrackNtuple/eventTree'].arrays()

In [52]:
data.fields

['trk_pt',
 'trk_charge',
 'trk_eta',
 'trk_phi',
 'trk_phi_local',
 'trk_d0',
 'trk_z0',
 'trk_chi2',
 'trk_chi2dof',
 'trk_chi2rphi',
 'trk_chi2rz',
 'trk_bendchi2',
 'trk_MVA1',
 'trk_nstub',
 'trk_lhits',
 'trk_dhits',
 'trk_seed',
 'trk_hitpattern',
 'trk_phiSector',
 'trk_genuine',
 'trk_loose',
 'trk_unknown',
 'trk_combinatoric',
 'trk_fake',
 'trk_matchtp_pdgid',
 'trk_matchtp_pt',
 'trk_matchtp_eta',
 'trk_matchtp_phi',
 'trk_matchtp_z0',
 'trk_matchtp_dxy',
 'trk_gtt_pt',
 'trk_gtt_eta',
 'trk_gtt_phi',
 'trk_gtt_selected_index',
 'trk_gtt_selected_emulation_index',
 'trkExt_pt',
 'trkExt_eta',
 'trkExt_phi',
 'trkExt_phi_local',
 'trkExt_d0',
 'trkExt_z0',
 'trkExt_chi2',
 'trkExt_chi2dof',
 'trkExt_chi2rphi',
 'trkExt_chi2rz',
 'trkExt_bendchi2',
 'trkExt_MVA',
 'trkExt_nstub',
 'trkExt_lhits',
 'trkExt_dhits',
 'trkExt_seed',
 'trkExt_hitpattern',
 'trkExt_phiSector',
 'trkExt_genuine',
 'trkExt_loose',
 'trkExt_unknown',
 'trkExt_combinatoric',
 'trkExt_fake',
 'trkExt_m

<Record ... trkHTEmuExt: 38.8} type='{"trk_pt": var * float32, "trk_charge": var...'>

In [74]:
data["trk_gtt_selected_emulation_index"]>=0

<Array [[True, True, True, ... True, False]] type='20284 * var * bool'>

In [62]:
data[data["trk_gtt_selected_emulation_index"]>=0]

ValueError: in ListArray64 attempting to get 151, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.5.0/src/cpu-kernels/awkward_ListArray_getitem_jagged_apply.cpp#L43)

In [76]:
gtt_cut = (data["trk_gtt_selected_emulation_index"]>=0)
data["trk_pt"][gtt_cut]

<Array [[4.01, 2.11, 2.32, ... 3.32, 2.35]] type='20284 * var * float32'>

In [106]:
gtt_cut = (data["trk_gtt_selected_emulation_index"]>=0)
input_names = ["trk_pt","trk_eta","trk_phi","trk_z0","trk_chi2rphi","trk_chi2rz","trk_bendchi2","trk_nstub"]
n_events = 200
n_trks = 100

X = np.empty((n_events, n_trks, len(input_names)),dtype=np.float32)
for ii in range(len(input_names)):
    feat = data[input_names[ii]][gtt_cut][:n_events]
    X[:, :, ii] = ak.to_numpy(ak.fill_none(ak.pad_none(feat, n_trks, axis=-1, clip=True),0,axis=-1))


In [107]:
model.build(X.shape)

In [108]:
tensor_X = K.constant(X)
output = model.call(tensor_X)

In [109]:
print(X.shape)
print(output.shape)

(200, 100, 8)
(200, 100, 5)


In [110]:
model.get_config()

{'name': 'find_phi',
 'trainable': True,
 'dtype': 'float32',
 'n_neighbours': 5,
 'n_dimensions': 5,
 'n_filters': 5,
 'n_propagate': 5,
 'also_coordinates': False,
 'feature_dropout': -1,
 'masked_coordinate_offset': None}